# An introduction to recursion schemes

## Part 1

Following [An introduction to recursion schemes (Part 1)](http://blog.sumtypeofway.com/an-introduction-to-recursion-schemes/)

In [1]:
{-# LANGUAGE DeriveFunctor #-}

data Lit = Str String
            deriving (Show, Eq)

data Expr a  
  = Index a a
  | Call a [a]
  | Unary String a
  | Binary a String a
  | Paren a
  | Literal Lit
  deriving (Show, Eq, Functor)
  
data Term f = In (f (Term f))

out :: Term f -> f (Term f)  
out (In t) = t

import Control.Arrow ((>>>),(<<<))

bottomUp :: Functor f => (Term f -> Term f) -> Term f -> Term f
bottomUp fn =  
  out                    -- 1) unpack: gets a `f (Term f)`
  >>> fmap (bottomUp fn) -- 2) recurse: get "inside" the `f` and apply `bottomUp fn` to it
  >>> In                 -- 3) repack: turn the transformed `f (Term f)` into a `Term a`
  >>> fn                 -- 4) apply: apply `bottomUp fn` to the final result

In [2]:
flattenT :: Term Expr -> Term Expr
flattenT (In (Paren e)) = e
flattenT other          = other

flatten :: Term Expr -> Term Expr
flatten = bottomUp flattenT

Defining a top down traversal is symmetrical:

1. apply
2. unpack
3. recurse
4. repack

In [3]:
topDown, bottomUp :: Functor f => (Term f -> Term f) -> Term f -> Term f 
bottomUp f = out >>> fmap (bottomUp f) >>> In  >>> f
topDown f  = In  <<< fmap (topDown f)  <<< out <<< f 

## Part II: A Mob of Morphisms

Following [Part II: A Mob of Morphisms](https://blog.sumtypeofway.com/recursion-schemes-part-2/)

In [4]:
{-# LANGUAGE DeriveFunctor #-}

data Expr a  
  = Literal { intVal :: Int }
  | Ident   { name :: String  }
  | Index   { target :: a, idx :: a }
  | Unary   { op :: String, target :: a }
  | Binary  { lhs :: a, op :: String, rhs :: a }
  | Call    { func :: a, args :: [a] }
  | Paren   { target :: a }
  deriving (Show, Eq, Functor)

In [5]:
newtype Term f = In { out :: f (Term f) }

In [6]:
ten, add, call :: Term Expr  
ten  = In (Literal { intVal = 10 })  
add  = In (Ident { name = "add" })  
call = In (Call { func = add, args = [ten, ten]}) -- add(10, 10)

> While this is a pleasing and concise representation of bottom-up transformations, it’s not as powerful as it could be: specifically, fn is limited to taking and returning a value of type `Term f`. We could not use `bottomUp` to count the total number of subexpressions in a tree (going from `Exprs` to `Ints`), nor could we transform this tree to a DOM representation (`Node`), nor could we render a term into a pretty-printed representation (`Doc`).

Going back to `bottomUp`:

In [7]:
bottomUp :: Functor f => (Term f -> Term f) -> Term f -> Term f
bottomUp fn =  
  out                    -- 1) unpack: gets a `f (Term f)`
  >>> fmap (bottomUp fn) -- 2) recurse: get "inside" the `f` and apply `bottomUp fn` to it
  >>> In                 -- 3) repack: turn the transformed `f (Term f)` into a `Term a`
  >>> fn                 -- 4) apply: apply `bottomUp fn` to the final result

What if we didn't repack in step 3?

Then `fn` must have type `f a -> a`

> This is a function type, taking as input a container `f` of values of type `a` and returning a bare value of type `a`.

What if we wanted to count the number of expressions?

```haskell
countNodes :: Expr Int -> Int
```

> The crucial element of understanding this function is its first parameter, here `Expr Int`. It captures an `Expr` **in the process of transformation** (...)

In [9]:
mystery fn =  
  out                   -- 1) unpack the Term
  >>> fmap (mystery fn) -- 2) recursively apply `fn`
  >>> fn                -- 3) apply `fn`
  
countNodes :: Expr Int -> Int
countNodes (Unary _ arg)         = arg + 1  
countNodes (Binary left _ right) = left + right + 1  
countNodes (Call fn args)        = fn + sum args + 1  
countNodes (Index it idx)        = it + idx + 1  
countNodes (Paren arg)           = arg + 1
countNodes (Literal _) = 1  --base case: no subexpression and fmap is identity here
countNodes (Ident   _) = 1  --base case: no subexpression and fmap is identity here

mystery countNodes call  

4

> (...) It’s almost magical how mystery “knows” how to stop recursing – but it lies in the definition of `fmap`. `fmap mystery` is the identity function over `Literal` and `Ident` values, as they do not contain any subexpressions. At this point, mystery stops recursing, applies the function `f`, and returns into its previous invocations. (...)

Los casos base de la recursion son las clausulas de `fmap` que no alteran el resultado.

## Catamorphisms

In [18]:
import Text.PrettyPrint (Doc)  
import qualified Text.PrettyPrint as P

type Algebra f a = f a -> a

-- (...) the “cata” in “catamorphism” is the same “cata” 
-- in “catastrophe”, “catabolism”, and “catalyst”—from 
-- the Greek κατα, meaning “downwards”, “into”, or “collapse”.
cata :: (Functor f) => Algebra f a -> Term f -> a  
cata f = out >>> fmap (cata f) >>> f

{-# LANGUAGE OverloadedStrings #-}

import Data.Monoid

prettyPrint :: Expr Doc -> Doc
prettyPrint (Literal i) = P.int i  
prettyPrint (Ident s) = P.text s
-- f(a,b...)  
prettyPrint (Call f as)     = f <> P.parens (P.cat (P.punctuate ", " as))  
-- a[b]
prettyPrint (Index it idx)  = it <> P.brackets idx  
-- op x
prettyPrint (Unary op it)   = P.text op <> it  
-- lhs op rhs
prettyPrint (Binary l op r) = l <> P.text op <> r  
-- (op)
prettyPrint (Paren exp)     = P.parens exp

cata prettyPrint call

add(10, 10)

> `bottomUp f` is just `cata f`, with the additional step of stuffing the accumulator value into a `Term` with `In` before handing it off to `f`

In [20]:
bottomUp f = cata (In >>> f)

## Anamorphisms

Applying the "reversing the arrows" trick we did to `topDown` to `cata`:

In [22]:
what f = In <<< fmap (what f) <<< f

:t what

what :: forall (f :: * -> *) a. Functor f => (a -> f a) -> a -> Term f

In [24]:
type Coalgebra f a = a -> f a

-- We call the what an anamorphism – the “ana” prefix, 
-- meaning “building”, is the opposite of “cata”
ana :: (Functor f) => Coalgebra f a -> a -> Term f  
ana f = In <<< fmap (ana f) <<< f